In [ ]:
pip install numpy

In [ ]:
pip install pandas

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import glob
import re

In [ ]:
files = glob.glob("Attendance Report/Raw data reports/*.csv")

In [ ]:
for f in files:
    # Load CSV
    df = pd.read_csv(f)

In [ ]:
all_dfs = []

In [ ]:
for f in files:
    # Read all columns as strings to avoid warnings
    df = pd.read_csv(f, dtype=str)

    # Extract Program Type
    program_type = "EHS" if "EHS" in f else "HS"

    # Extract Program Year
    year_match = re.search(r"(\d{4}\s*-\s*\d{4})", f)
    program_year = year_match.group(1) if year_match else None

    df["ProgramType"] = program_type
    df["ProgramYear"] = program_year
    df["SourceFile"] = f.split("/")[-1]

    all_dfs.append(df)


In [ ]:
full_df = pd.concat(all_dfs, ignore_index=True)

full_df.head()

In [ ]:
full_df.shape

In [ ]:
full_df.columns

In [ ]:
full_df["AttendanceDate"] = pd.to_datetime(full_df["Attendance Date"], errors="coerce")
full_df["EligibilityIncome"] = pd.to_numeric(full_df["Eligibility Income"], errors="coerce")

In [ ]:
full_df.tail()

In [ ]:
full_df.columns = (
    full_df.columns
    .str.strip()
    .str.replace(" ", "_")
    .str.replace(r"[^A-Za-z0-9_]", "", regex=True)
)

In [ ]:
full_df["Attendance_Date"] = pd.to_datetime(full_df["Attendance_Date"], errors="coerce")
full_df["Enrolled_Date"]   = pd.to_datetime(full_df.get("Enrolled_Date"), errors="coerce")

In [ ]:
full_df["IsAbsent"]  = (full_df["Attendance_Code_Description"] == "Absent").astype(int)
full_df["IsPresent"] = (full_df["Attendance_Code_Description"] == "Present").astype(int)

In [ ]:
def clean_binary(colname):
    if colname not in full_df.columns:
        return
    full_df[colname] = (
        full_df[colname]
        .astype(str)
        .str.strip()
        .str.capitalize()
        .replace({"Y": "Yes", "N": "No", "Nan": np.nan, "": np.nan})
    )

In [ ]:
for c in [
    "Homeless_Family",
    "Lives_With_Family",
    "Transportation_Services_Needed",
    "Transportation_Services_Received",
]:
    clean_binary(c)

In [ ]:
group_keys = ["Name_Last", "Name_First", "ProgramYear","Site_Name"]

summary = full_df.groupby(group_keys).agg(
    Total_Days   = ("IsAbsent", "count"),
    Absent_Days  = ("IsAbsent", "sum"),
    Present_Days = ("IsPresent", "sum"),
    Absence_Rate = ("IsAbsent", "mean"),
).reset_index()

# Add demographics (taking first non-null value per student/year)
demo_cols = [
    "Homeless_Family",
    "Lives_With_Family",
    "Income_Status_Code_Description",
    "Eligibility_Income",
    "Transportation_Services_Needed",
    "Transportation_Services_Received",
    "ProgramType"
]

demo_df = (
    full_df[group_keys + [c for c in demo_cols if c in full_df.columns]]
    .drop_duplicates(subset=group_keys, keep="first")
)

summary = summary.merge(demo_df, on=group_keys, how="left")

summary.head()

In [ ]:
def clean_numeric(col):
    return (
        col.astype(str)
        .str.replace(",", "", regex=False)
        .str.extract(r"(\d+)", expand=False)  # keep only digits
        .pipe(pd.to_numeric, errors="coerce")
    )

In [ ]:
summary["Eligibility_Income_num"] = clean_numeric(summary["Eligibility_Income"])

In [ ]:
#Group attendance by income quartiles
summary.groupby(pd.qcut(summary["Eligibility_Income_num"], q=4, duplicates="drop"))[
    "Absence_Rate"
].mean()

In [ ]:
#How homelessness affects attendance
homeless_metrics = summary.groupby("Homeless_Family").agg(
    Students = ("Name_Last", "count"),
    Avg_Absence_Rate = ("Absence_Rate", "mean"),
    Median_Absence_Rate = ("Absence_Rate", "median"),
    Avg_Absent_Days = ("Absent_Days", "mean"),
    Avg_Total_Days = ("Total_Days", "mean")
).reset_index()

homeless_metrics

In [ ]:
homeless_metrics["Avg_Absence_Rate_Pct"] = homeless_metrics["Avg_Absence_Rate"] * 100
homeless_metrics


In [ ]:
#Lives with family or not
lives_metrics = summary.groupby("Lives_With_Family").agg(
    Students = ("Absence_Rate", "count"),
    Avg_Absence_Rate = ("Absence_Rate", "mean"),
    Median_Absence_Rate = ("Absence_Rate", "median"),
    Avg_Absent_Days = ("Absent_Days", "mean"),
    Avg_Total_Days = ("Total_Days", "mean")
).reset_index()

# Convert to percentage
lives_metrics["Avg_Absence_Rate_Pct"] = lives_metrics["Avg_Absence_Rate"] * 100

lives_metrics


In [ ]:
#Club all the public assistance into one as only 2 years of reports have
summary["Income_Status_Grouped"] = summary["Income_Status_Code_Description"]

summary["Income_Status_Grouped"] = summary["Income_Status_Grouped"].replace({
    "Public Assistance": "Public Assistance (Combined)",
    "SSI (Public Assistance)": "Public Assistance (Combined)",
    "SNAP (Public Assistance)": "Public Assistance (Combined)",
    "TANF (Public Assistance)": "Public Assistance (Combined)"
})


In [ ]:
#Based on Income Status Code Description
income_metrics = summary.groupby("Income_Status_Grouped").agg(
    Students = ("Absence_Rate", "count"),
    Avg_Absence_Rate = ("Absence_Rate", "mean"),
    Median_Absence_Rate = ("Absence_Rate", "median"),
    Avg_Absent_Days = ("Absent_Days", "mean"),
    Avg_Total_Days = ("Total_Days", "mean")
).reset_index()

# Convert to cleaner percentage form
income_metrics["Avg_Absence_Rate_Pct"] = income_metrics["Avg_Absence_Rate"] * 100

income_metrics.sort_values("Avg_Absence_Rate", ascending=False)


In [ ]:
#Monthly trends
full_df["Attendance_Date"] = pd.to_datetime(full_df["Attendance_Date"], errors="coerce")
full_df["Month"] = full_df["Attendance_Date"].dt.month
full_df["Year"] = full_df["Attendance_Date"].dt.year


In [ ]:
monthly_absence = (
    full_df.groupby(["ProgramYear", "Month"])["IsAbsent"]
    .mean()
    .reset_index()
    .sort_values(["ProgramYear", "Month"])
)


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12,6))

for py, grp in monthly_absence.groupby("ProgramYear"):
    plt.plot(
        grp["Month"],
        grp["IsAbsent"],
        marker="o",
        label=str(py)
    )

plt.xticks(range(1,13))
plt.xlabel("Month")
plt.ylabel("Absence Rate")
plt.title("Monthly Absence Trends by Program Year")
plt.legend(title="Program Year")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
full_df["Season"] = full_df["Month"].apply(
    lambda m: "Winter" if m in [12, 1,] else "Non-Winter"
)

seasonal_stats = full_df.groupby("Season")["IsAbsent"].mean() * 100
seasonal_stats


In [ ]:
# Ensure Month field exists
full_df["Month"] = full_df["Attendance_Date"].dt.month

# Compute monthly absence rate
monthly_abs = full_df.groupby("Month")["IsAbsent"].mean()

# Convert to attendance rate
monthly_attendance = 1 - monthly_abs

monthly_attendance


In [ ]:
attendance_ranking = monthly_attendance.sort_values(ascending=False)
attendance_ranking


In [ ]:
#Site specific analysis

In [ ]:
sites_2526 = (
    full_df.loc[full_df["ProgramYear"]=="2025 - 2026", "Site_Name"]
    .dropna()
    .unique()
)
sites_2526


In [ ]:
full_df["Site_Name"] = full_df["Site_Name"].replace({
    "EHS Nadaburg": "BTF Nadaburg",
    "Nadaburg EHS": "BTF Nadaburg"   # in case other variations appear
})


In [ ]:
#Filter only the current active sites
active_df = full_df[full_df["Site_Name"].isin(sites_2526)].copy()
active_df.shape


In [ ]:
summary.columns

In [ ]:
#Filter only the current active sites
summary = summary[summary["Site_Name"].isin(sites_2526)].copy()
summary.shape

In [ ]:
# Student-level summary (site + student)
site_student_summary = active_df.groupby(
    ["Site_Name", "Name_Last", "Name_First", "ProgramYear"]
).agg(
    Total_Days=("IsPresent", "count"),
    Absent_Days=("IsAbsent", "sum"),
    Present_Days=("IsPresent", "sum"),
    Absence_Rate=("IsAbsent", "mean")
).reset_index()
site_student_summary

In [ ]:
site_metrics = site_student_summary.groupby("Site_Name").agg(
    Students=("Absence_Rate", "count"),
    Avg_Absence_Rate=("Absence_Rate", "mean"),
    Median_Absence_Rate=("Absence_Rate", "median"),
    Avg_Absent_Days=("Absent_Days", "mean"),
    Avg_Total_Days=("Total_Days", "mean")
).reset_index()

site_metrics["Avg_Absence_Rate_Pct"] = site_metrics["Avg_Absence_Rate"] * 100

site_metrics.sort_values("Avg_Absence_Rate", ascending=False)


In [ ]:
full_df["Attendance_Date"] = pd.to_datetime(full_df["Attendance_Date"], errors="coerce")
full_df["Month"] = full_df["Attendance_Date"].dt.month

In [ ]:
site_metrics = site_student_summary.groupby("Site_Name").agg(
    Students=("Absence_Rate", "count"),
    Avg_Absence_Rate=("Absence_Rate", "mean"),
    Median_Absence_Rate=("Absence_Rate", "median"),
    Avg_Absent_Days=("Absent_Days", "mean"),
    Avg_Total_Days=("Total_Days", "mean"),
).reset_index()

site_metrics["Attendance_Rate"] = 1 - site_metrics["Avg_Absence_Rate"]
site_metrics["Avg_Absence_Rate_Pct"] = site_metrics["Avg_Absence_Rate"] * 100
site_metrics["Attendance_Rate_Pct"] = site_metrics["Attendance_Rate"] * 100

site_metrics.head()

In [ ]:
# Filter to absences only (so denominator is "all absence days")
absent_only = active_df[active_df["IsAbsent"] == 1].copy()

# Transport problem subset
tp_df = absent_only[
    absent_only["Absence_Reason_Code_Description"] == "Transportation Problem"
]

# Total absences per site
site_total_absences = absent_only.groupby("Site_Name")["IsAbsent"].count()

# Transport absences per site
site_tp_absences = tp_df.groupby("Site_Name")["IsAbsent"].count()

transport_metrics = (
    pd.DataFrame({
        "Total_Absences": site_total_absences,
        "Transport_Absences": site_tp_absences
    })
    .fillna(0)
)

transport_metrics["Transport_Absence_Share"] = (
    transport_metrics["Transport_Absences"] / transport_metrics["Total_Absences"]
)

transport_metrics = transport_metrics.sort_values("Transport_Absence_Share", ascending=False)
transport_metrics


In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(transport_metrics.index, transport_metrics["Transport_Absence_Share"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Share of Absences Due to Transportation")
plt.title("Transportation-Related Absences by Site (Active Sites Only)")
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Filter to "Unable to Contact Family" absences
utc_df = absent_only[
    absent_only["Absence_Reason_Code_Description"] == "Unable to Contact Family"
]

site_utc_absences = utc_df.groupby("Site_Name")["IsAbsent"].count()

utc_metrics = (
    pd.DataFrame({
        "Total_Absences": site_total_absences,
        "UTC_Absences": site_utc_absences
    })
    .fillna(0)
)

utc_metrics["UTC_Absence_Share"] = (
    utc_metrics["UTC_Absences"] / utc_metrics["Total_Absences"]
)

utc_metrics = utc_metrics.sort_values("UTC_Absence_Share", ascending=False)
utc_metrics


In [ ]:
plt.figure(figsize=(12, 6))
plt.bar(utc_metrics.index, utc_metrics["UTC_Absence_Share"])
plt.xticks(rotation=45, ha="right")
plt.ylabel("Share of Absences: 'Unable to Contact Family'")
plt.title("'Unable to Contact Family' Absences by Site (Active Sites Only)")
plt.grid(axis="y", alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
reason_month = (
    active_df[active_df["IsAbsent"] == 1]
    .groupby(["Absence_Reason_Code_Description", "Month"])
    .size()
    .reset_index(name="Count")
)

In [ ]:
full_df["Income_Status_Grouped"] = full_df["Income_Status_Code_Description"]

full_df["Income_Status_Grouped"] = full_df["Income_Status_Grouped"].replace({
    "Public Assistance": "Public Assistance (Combined)",
    "SSI (Public Assistance)": "Public Assistance (Combined)",
    "SNAP (Public Assistance)": "Public Assistance (Combined)",
    "TANF (Public Assistance)": "Public Assistance (Combined)"
})


In [ ]:
active_df = full_df[full_df["Site_Name"].isin(sites_2526)].copy()

In [ ]:
def subgroup_breakdown(reason):
    df = active_df[
        (active_df["IsAbsent"] == 1) &
        (active_df["Absence_Reason_Code_Description"] == reason)
    ]
    return df["Income_Status_Grouped"].value_counts(normalize=True) * 100


In [ ]:
subgroup_breakdown("Sick")

In [ ]:
subgroup_breakdown("Transportation Problem")

In [ ]:
subgroup_breakdown("Unable to Contact Family")

In [ ]:
homeless_df = active_df[
    (active_df["IsAbsent"] == 1) &
    (active_df["Homeless_Family"] == "Yes")
]


In [ ]:
homeless_reasons = (
    homeless_df["Absence_Reason_Code_Description"]
    .value_counts()
)

homeless_reasons_pct = (
    homeless_reasons / homeless_reasons.sum() * 100
)


In [ ]:
pd.DataFrame({
    "Count": homeless_reasons,
    "Percent": homeless_reasons_pct
})


In [ ]:
#active_df.to_csv("Complete_dataframe.csv", index=False)


In [ ]:
#summary.to_csv("summary_dataframe.csv", index=False)

In [ ]:
active_df.columns

In [ ]:
summary.columns